In [ ]:
'''
Title: Bigram LLM Model
Author: Siddharth Sharma
Date: Aug, 24,2024 -- PST
++ Update 1: Aug, 25, 2024 -- PST
Discription: This notbook contains basics for the LLM Project. - A Bigram Language Model.
--
To_deploy: Streamlit
--

References:
Step by Step  vedio:
https://www.freecodecamp.org/news/how-to-build-a-large-language-model-from-scratch-using-python/ To Follow this video
https://github.com/ajitsingh98/LLMs-From-Scratch-In-PyTorch

Book / Reading:
https://github.com/ManalAbumelhaa/LLM-LLMs-from-scratch

'''

'\nTitle: Birgam LLM Model\nAuthor: Siddharth Sharma\nDate: Aug, 24,2024 -- PST\nDiscription: This notbook contains basics for the LLM Project. - A Bigram Language Model.\n--\nTo_deploy: Streamlit\n--\n\nReferences:\nStep by Step  vedio:\nhttps://www.freecodecamp.org/news/how-to-build-a-large-language-model-from-scratch-using-python/ To Follow this video\nhttps://github.com/ajitsingh98/LLMs-From-Scratch-In-PyTorch\n\nBook / Reading:\nhttps://github.com/ManalAbumelhaa/LLM-LLMs-from-scratch\n\n'

In [ ]:
# iinstall the compression algo
!pip install pylzma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylzma: filename=pylzma-0.5.0-cp310-cp310-linux_x86_64.whl size=222322 sha256=5ac300d94b2c0e6c467e192c24ff7acf3589669afeba6766765f6a2e7f1d5137
  Stored in directory: /root/.cache/pip/wheels/74/c9/02/91112815e838f544c1d46fda071241e454694579d022751d2b
Successfully built pylzma


In [ ]:
import pylzma
import torch
import torch.nn as nn
from torch.nn import functional as F

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open ("/content/drive/MyDrive/Colab Notebooks/From Scratch/Data Sets/wizard_of_oz.txt" , 'r' , encoding = 'utf-8') as f:
  text = f.read()

In [ ]:
print(text[:300]) # if we do simply text[:300] it will not be visually appealing as it will also have the newline characters.


Title: The Wonderful Wizard of Oz

Author: L. Frank Baum

Release date: February 1, 1993 [eBook #55]
                Most recently updated: March 30, 2021

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK THE WONDERFUL WIZARD OF OZ ***

[Illustration]




The Wonderful Wizard of Oz

by


In [82]:
len(text)

208087

We now need to build a tokenizer, wherein we will see how many characters do the whole text contains and this tokenizer will contain of a encoder and a decoder, each character will be assigned a number or a  (HASH)


This is a character level tokenizer,

there are other - sub word level, sentence level and word level tokenizer's

In [ ]:
chars = sorted(set(text))
print(chars) # this contains all the different characters present in the text.
print(len(chars))

['\n', ' ', '!', '#', '&', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '5', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']
79


In [84]:
string_to_int = {ch:i for i,ch in enumerate(chars)} # in dict format
int_to_string = {i:ch for i,ch in enumerate(chars)}

# print(string_to_int)
# print(int_to_string)

encode = lambda s:[string_to_int[c] for c in s] # keeping it simple here, but better encoding  will make the custom code more secure...
decode = lambda l: ''.join([int_to_string[i] for i in l])

# .get(i,'')

# sample to see if the intended code working or not
# print(encode("testing"))
# print(encode("Testing"))
# print(decode(encode("Testing")))

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:50])

tensor([ 0, 39, 56, 67, 59, 52, 17,  1, 39, 55, 52,  1, 42, 62, 61, 51, 52, 65,
        53, 68, 59,  1, 42, 56, 73, 48, 65, 51,  1, 62, 53,  1, 34, 73,  0,  0,
        20, 68, 67, 55, 62, 65, 17,  1, 31, 10,  1, 25, 65, 48])


torch.long - we are going to create a tensor and we will have the tensor conaitn super long integers

we will be storign our data in tensors now.

Now once the data is defined in tensor's we will be splitting the data into training and testing split. the text corpus now. (70/30) split since the data corpus is small.

Link of the chat: https://chatgpt.com/share/6d4cb86a-e766-4796-8c74-63381191184d

***Bigram Models***

 bigram model is a type of statistical language model used in natural language processing (NLP) that predicts the probability of a word given the previous word in a sequence. In simpler terms, it considers pairs of consecutive words (known as "bigrams") to understand the likelihood of a word following another in a sentence.

How it works:
Sequence of Words: The bigram model breaks down a text into pairs of consecutive words.
For example, in the sentence "I am learning NLP," the bigrams would be ("I", "am"), ("am", "learning"), and ("learning", "NLP").
Probability Calculation: The model calculates the probability of a word given the previous word,


appears as the first word in any bigram in the text.
Application: This model is used to predict the next word in a sequence, generate text, or assess the likelihood of a given sequence of words. It's relatively simple but provides a foundational understanding of word dependencies in text.
Limitations:
Context: The bigram model only considers the previous word, so it doesn't capture long-range dependencies or context beyond one word back.
Data Sparsity: If certain bigrams are rare or unseen in the training data, the model might struggle to predict them accurately.
Overall, while the bigram model is a fundamental and straightforward approach, it is often extended to more complex models like trigrams (which consider three-word sequences) or even more sophisticated models like neural networks that capture broader context.



In [85]:
# creating a test train split:
# 80 percent training
threshold= 0.8*len(data)

train_text = data[:int(threshold)]
test_text = data[int(threshold):]


In [86]:
block_size = 8

x = train_text[:block_size]
y = train_text[1:block_size+1] # increamenting the character by 1, to see whether if it changes the proba of the otucomes.

for block_word in range (block_size):
  context = x[:block_word +1] # the context will keep on increasing, as the for loops progress, at the end of the loop the context will be the same size of 8 block words (here) (or the prompt data])
  target = y[block_word] #
  print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 39
when input is tensor([ 0, 39]) the target: 56
when input is tensor([ 0, 39, 56]) the target: 67
when input is tensor([ 0, 39, 56, 67]) the target: 59
when input is tensor([ 0, 39, 56, 67, 59]) the target: 52
when input is tensor([ 0, 39, 56, 67, 59, 52]) the target: 17
when input is tensor([ 0, 39, 56, 67, 59, 52, 17]) the target: 1
when input is tensor([ 0, 39, 56, 67, 59, 52, 17,  1]) the target: 39


Now that the block size is defined as 8 and the increment as 1, we  want to scale it up and hence we are wanting to process this in batches: (easist wyay to scale up and perform deep learningnlp tasks)

I'd be using the basic GPU T4 - Google collab easily available for a bit of faster processing, else CPU

In [88]:
tensor_1 = torch.tensor([1.,2.,3.])

# get probas / Apply softmax
softmax_output = F.softmax(tensor_1, dim=0)

# NOTE: Why dim=0?
# In this context, dim=0 is used because tensor_1 is a 1-dimensional tensor.
# If it were a multi-dimensional tensor, dim would control along which dimension the softmax operation is applied.
# Since there's only one dimension here, dim=0 is appropriate.
# If you were working with a 2D tensor (e.g., a matrix), specifying dim=0 would apply the softmax across columns,
# while dim = 1 would apply it across rows.


print(softmax_output)

tensor([0.0900, 0.2447, 0.6652])


Todo: Learn a bit more about the nn.embeddings layering nd how this embedding is done.

In [66]:
# # Initialize an embedding layer
# vocab_size = 10000
# embedding_dim = 100
# embedding = nn.Embedding(vocab_size, embedding_dim)
# # Create some input indices
# input_indices = torch.LongTensor([1,5, 3, 2])
# # Apply the embedding layer

# embedded_output = embedding(input_indices)
# # The output will be a tensor of shape (4, 100), where 4 is the number of input
# # and 100 is the dimensionality of the embedding vectors

# print(embedded_output.shape)
# embedded_output_with_nan = torch.where(embedded_output > 1, embedded_output, torch.tensor(np.nan))
# print(embedded_output_with_nan)

Matmul

In [67]:
# a = torch.tensor([[1,2],[3,4],[5,6]])
# b = torch.tensor([[2,8,4],[10,11,12]])
# print(a @ b)
# @ or matmul is the same {kinda}, but in pytorch you can not multiply int and floats.

We will now make a bigram model, and we will be keeping a size of 8 for the block. this is a character level bigram model and here the increament of the block "characters" will be by 1 (block size remaining the same)

In [106]:
# making different env / Choosing Cuda for batch processing:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # just checks if gpu is available or not (here T4)
print(device)

# Constants:
# hyper parameters
block_size = 8 # the basic bigram length of the sequence.
batch_size = 4 # for parallel batch processing the tensors.
max_iters = 10000 # kinda like epochss uoi'll see i in the later code.
eval_interval = 250
learning_rate = 3e-4 # kept it pretty small
dropout = 0.2 # while training the model, this hhyperparameter will drop random neuraon / perceptrons from the model so that the model does not over fit the data.
vocab_size = len(chars) # Very important, this is not learnable parameter, but is very important to be defined dyanmically else there will be an KeyError.

cpu


In [111]:
# Batch code:

def get_batch(split):
  data = train_text if split == 'train' else test_text
  i_x = torch.randint(len(data) - block_size, (batch_size,))
  # print(len(data) - block_size, (batch_size,))
  # print(len(data)) # why ?
  # print(i_x)
  # print(i_x.shape)
  x = torch.stack([data[i:i+block_size] for i in i_x])
  y = torch.stack([data[i+1:i+block_size+1] for i in i_x])
  x,y = x.to(device), y.to(device) # Cuda here -  to process the data in batches.
  # in CPU the processing would be in sequence whic will take a lot of time and the
  # context windows might also suffer, in better / advanced models.
  return x,y



In [92]:
  # print(train_text.shape)
  x,y = get_batch('train')
  print('inputs:')
  print(x)
  print('targets:')
  print(y)

166461 (4,)
tensor([108264, 119040,  60086,  46522])
inputs:
tensor([[52,  1, 70, 52, 65, 52,  1, 67],
        [61,  1, 51, 62, 56, 61, 54,  1],
        [42, 55, 48, 67,  1, 48, 65, 52],
        [ 1, 48, 61, 71, 56, 62, 68, 66]])
targets:
tensor([[ 1, 70, 52, 65, 52,  1, 67, 55],
        [ 1, 51, 62, 56, 61, 54,  1, 67],
        [55, 48, 67,  1, 48, 65, 52,  1],
        [48, 61, 71, 56, 62, 68, 66,  1]])


In [93]:

@torch.no_grad() # a decorater wherre hte gradienti computation will not happen to save us form a lot of computation
def estimate_loss():
    out = {}
    model.eval() # (dropout has to be turned off here, so that the model can accuratley follow whatever the model has learnt.)
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # passing the model here, put the model in teh training mode. (dropout is active here to help the model not overfit the data)
    return out


# Batch
Batch refers to the number of sequences (or samples) processed simultaneously during the forward pass of the model. Processing data in batches is a common practice in deep learning to take advantage of parallelism, which speeds up training and allows for more efficient use of memory.


# Time
Time refers to the length of the sequence of tokens (characters) being input to the model. In the case of text data, this would correspond to the number of characters or tokens in a given sequence. For example, if you're inputting a sentence of 5 characters, the Time dimension would be 5.
Why is Time needed?: The Time dimension is essential because it allows the model to learn dependencies between tokens across a sequence. By looking at multiple tokens in sequence, the model can learn that certain tokens are more likely to follow others, which is crucial for tasks like language modeling. Even in a bigram model (which only considers pairs of tokens), the sequence length (Time) helps in maintaining context within the batch.


# Channel
Channel corresponds to the number of possible outputs at each time step, which in this case is the size of the vocabulary. This dimension represents the probabilities of each token in the vocabulary being the next token in the sequence.
After passing through the embedding layer, the shape of the logits tensor is (Batch, Time, Channel), where:
  - Batch is the number of sequences.
  - Time is the length of each sequence.
  - Channel (vocab size) is the number of possible tokens that could follow each token in the sequence.

In [102]:

class BigramLanguageModel(nn.Module):
    """
    A simple Bigram Language Model built using PyTorch's neural network module (nn.Module).
    This model learns the probabilities of one token (character) following another in a sequence.(This is not for long context windows.)
    """

    def __init__(self, vocab_size):
        """
        Initialize the model with the given vocabulary size.

        Args:
            vocab_size (int): The size of the vocabulary, i.e., the number of unique tokens (characters) in the data.
        """
        super().__init__()  # Initialize the parent class (nn.Module)
        # Create an embedding table where each token is represented by a vector of size vocab_size
        # This table will store the probabilities of each token following another token
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, indices, targets=None):
        """
        Perform the forward pass through the model.

        Args:
            indices (Tensor): A tensor of shape (Batch, Time) containing indices of the input tokens.
            targets (Tensor, optional): A tensor of shape (Batch, Time) containing the indices of the target tokens.
                                         If provided, the loss will be computed; otherwise, only logits are returned.

        Returns:
            logits (Tensor): The raw predictions from the model (before applying softmax).
            loss (Tensor or None): The cross-entropy loss if targets are provided, otherwise None.
        """
        # Look up the embedding for each token in the input sequence
        logits = self.token_embedding_table(indices)  # (Batch, Time, Channel) where Channel = vocab_size

        # If targets are provided, calculate the loss
        if targets is None:
            loss = None
        else:
            Batch, Time, Channel = logits.shape
            # Reshape logits and targets to match the expected input shape for cross-entropy loss
            logits = logits.view(Batch * Time, Channel)
            targets = targets.view(Batch * Time)
            # Calculate the cross-entropy loss between logits and targets
            loss = F.cross_entropy(logits, targets)

        return logits, loss
        # why time ?

    def generate(self, indices, max_new_tokens):
        """
        Generate new tokens based on the input sequence.

        Args:
            indices (Tensor): A tensor of shape (Batch, Time) containing the starting sequence of tokens.
            max_new_tokens (int): The number of new tokens to generate.

        Returns:
            indices (Tensor): A tensor containing the original and newly generated tokens.
        """
        for _ in range(max_new_tokens):
            # Get predictions from the model
            logits, _ = self.forward(indices) # be explicit, this is a forward pass, optimizers work usually in the backward pass, and re-revise the weights and the biases if any.
            # TODO: will be using AdamW or Adam later on, (just a reminder, when you read this, adam does use momentum and adjustable alpha.... (i forgot the rest, revise the opt again))
            # Apply softmax to convert logits to probabilities
            probs = F.softmax(logits[:, -1, :], dim=-1)
             # We focus on the last token in the sequence for prediction
            # Sample from the probability distribution to get the next token
            index_next = torch.multinomial(probs, num_samples=1)  # (Batch, 1)
            # torch.multinomial: https://pytorch.org/docs/stable/generated/torch.multinomial.html
            '''
              torch.multinomial
              torch.multinomial is a function in PyTorch used to sample elements from a given distribution.
              It is particularly useful when you want to draw samples based on probabilities,
              rather than just selecting the most probable element.

              probs: This is a tensor of shape (Batch, vocab_size) that represents the
              probability distribution over the vocabulary for the next token.

              The result, index_next, is a tensor of shape (Batch, 1),
              where each element is the index of the sampled token based on the model's predicted probabilities.

            '''
            # Append the newly generated token to the input sequence
            indices = torch.cat((indices, index_next), dim=1)  # (Batch, Time+1)

        return indices


In [101]:
# Initialize the model
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# Starting context tensor (batch_size=1, sequence_length=1)
context = torch.zeros((1, 1), dtype=torch.long, device=device) # torch.long is equivalent to the int64

# Generate new tokens (in the sequence of 500 tokens)
generated_indices = model.generate(context, max_new_tokens=500)[0].tolist() # the max new tokens in GPT's are autogenerated and take an optimised learning purview.

generated_chars = decode(generated_indices)
print(generated_chars)


q;;u—WHoDy;ZKgNR 2i““RhJ]MNI)*kC)&dYnyX&ccQ‘m.x;!A
qAKCJHvA2ZZwI:xMMN#LPnaD&Ly-IN1gcQvV;.QkTH*P2n‘wDR[ymixqC)‘fCpuLg
r&0o25zx?M1pomSVpM”D‘VtZdNl’DQTIhfW?Ya CZcPcV]KjCPnJh)ObW2ZD‘h1#LhkbnrCu’n;)“(& *okBZVh)qLPSG?vAoKIVhD50puT,E“X“&‘5‘J;0P&V;xnu0
l2GpJZ”Tv”YtsGC)‘lfXd;”Q—SS&k0Z—rVXhD(SVhujTELhlE2crg2]&ABb RE[IIHHAymPrPB.X‘“mx5PbSB*Zp# nY5e]A]bXnyEpS2roJuT’iHIB”y0PZP5o—E9HO)*dQsx 2ZVeTj ;0o‘?JffEBU?wKZ,dI#LcFZ&iTYn5vw]o
tqjYxNQjKkC“GPnKOuAcp“glbW
qB;[bbTlZhd?Vx.hII?#IQ2YBP—XeS25A!2SpM(U&2,an Cscykz


As this is a character level tokenizer, hence it just might seem that it blurts out stuff that makes little to no sense, but this is based on logits and cross-entropy, the first character that is being generated will have the lowest loss and the hightest frequency of all the characters - (here 79 unique characters)


we can see the here 'q' is the first character, and we did have  qwerty keyboards for a reason.  --- Sid

# Optimisations
- of the learnable parameters
- AdamW here :)

In [107]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
'''
      AdamW is a type of optimizer that combines the benefits of both Adam (Adaptive Moment Estimation) and weight decay (L2 regularization).
      It helps in finding the optimal weights for the model efficiently.


      Why AdamW?
      Pros:
      Adaptive Learning Rates: AdamW adjusts the learning rate for each parameter individually,
      which can help the model converge faster and more reliably.

      Weight Decay: Helps prevent overfitting by penalizing large weights,
      which is crucial in reducing the model's tendency to memorize the training data.

      Cons:
      Computationally Intensive: AdamW requires more memory and computational power compared to simpler
      optimizers like SGD (Stochastic Gradient Descent).
      Sensitivity to Hyperparameters: The performance of AdamW can be sensitive to the choice of learning rate and other hyperparameters.


'''

for i in range(max_iters):
  xb, yb = get_batch('train') # data, targets for the training model

# periodic evaluations (in set number of iterations)
  if iter % eval_iters == 0: # in every 250 iterations
    losses = estimate_loss()
    print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}") # logging the evals
  '''
        This is where iter % eval_iters == 0 comes in. This condition checks if the current iteration number is divisible by eval_iters. It's a common technique to perform periodic evaluations or logging without doing it every single iteration.

      - If true, it estimates the loss (presumably on both training and validation sets) and prints the results.
      - This allows you to monitor the model's performance at regular intervals during training.

  '''

  # evaluate the loss
  logits, loss = model.forward(xb,yb) # forward pass.
  optimizer.zero_grad(set_to_none = True) # usually set as False for Rnn's as they need to remember the context. (also try avg_zero_grad, when in rnn's)
  '''
      Forward Pass:
      The model processes the input data xb and generates predictions (logits).
      The logits are the raw scores before applying an activation function like softmax.

      Loss Calculation: The loss function compares the model's predictions with the actual target values (yb)
      and calculates how far off the predictions are. The lower the loss, the better the model is performing.

      Why Loss?
      Loss Function: The loss function provides a single scalar value that the optimizer uses to update the model's weights.
      Cons: A poor choice of loss function or improper tuning can lead to suboptimal model performance.
        '''
        """
      Why Zero Gradients?
      Gradients are the derivatives of the loss with respect to the model's weights.
      If you don’t reset them to zero, they accumulate over time, leading to incorrect updates.
      Set to None: This is a more efficient way to reset the gradients as it reduces memory consumption.

      Pros:
      Accuracy: Ensures that the current update is based solely on the current batch of data.
      Efficiency: Helps prevent unnecessary memory usage.

  """


  # the previous gradient does not add overtime, since this is a bigram models we do not want our previous. (long) data to influence the result of our current data
  # the max iters  (was earlier 1000, but now is 10000) and the loss is even. though less it is more than 1 (and we are using probas and logits) - this means that this model will perform ery poorly if there is a temperature parameter set here.
  # we got the loss to be about ~~ 3.008 pretty poor.
  # increasing the iter's might also overfit the data.
  loss.backward()

  """
      Backward Pass:
      The backward pass calculates the gradient (or derivative) of the loss with respect to each parameter in the model.
       This is crucial for the optimizer to know how to adjust the weights to reduce the loss.

      Why Backpropagation?
      Gradient Descent: Backpropagation is an essential part of the gradient descent algorithm, which is used to minimize the loss function.
      Cons: Complexity: Can be computationally expensive, especially for deep networks.

  """
  optimizer.step() # it updates the model parameter/s based on the computed gradients. (weights biases,etc)
  # this also contains the stopping condition and will stop where the loss will be the least.
  # next step till the optimizations is reached, or think of it as the step in the right direction ... lol


  print(loss.item())


Streaming output truncated to the last 5000 lines.
166461 (4,)
tensor([ 33454,  33231, 138310, 161404])
166461 (4,)
tensor([ 39086,  37181, 163709,  88136])
166461 (4,)
tensor([ 26771,   5193,  92517, 165778])
166461 (4,)
tensor([  5486,  22123, 114737,  14272])
166461 (4,)
tensor([147540,  43562,  46913,  40565])
166461 (4,)
tensor([116459,  13836, 139569,  44409])
166461 (4,)
tensor([51910, 84556, 65278, 19430])
166461 (4,)
tensor([ 46164,  75098, 108904, 118840])
166461 (4,)
tensor([ 91944, 117432, 163969,  57786])
166461 (4,)
tensor([ 75351, 128679,  27454,  50250])
166461 (4,)
tensor([ 17783, 112157, 137297, 131867])
166461 (4,)
tensor([87790, 69854, 61499,   820])
166461 (4,)
tensor([ 20741, 137010,  73966,    488])
166461 (4,)
tensor([ 22948,  47069, 107673,  22059])
166461 (4,)
tensor([101830,  95373,   4196, 159095])
166461 (4,)
tensor([46881,   942, 63949, 71123])
166461 (4,)
tensor([156607, 100999,  81625, 152381])
166461 (4,)
tensor([ 72362,  24376, 107198, 104103])
166461 

In [112]:

# Starting context tensor (batch_size=1, sequence_length=1)
context = torch.zeros((1, 1), dtype=torch.long, device=device) # torch.long is equivalent to the int64

# Generate new tokens (in the sequence of 500 tokens)
generated_indices = model.generate(context, max_new_tokens=500)[0].tolist()

generated_chars = decode(generated_indices)
print(generated_chars)
# pretty grabage tokens, but this is because we have a character level token here, and the embedding table does not have any meaningful relationships.



HBbe?g Wwoy;zjzD‘V, caino1A&F—tp[KStZhidslfugeyw.pJctc0o
0PB I-I’&lererjF. a1gedQ225vjVGpFVGw1g.F)a—bwishrookbL--wlxhr. 5HerDBZlff0JedFv.sawnagoofo.roke

LVR].pr. Oqter. skF,
”nsEJZs tSVw my’e t
0d:EGzBR
nsm twnangausw.SK#.jBPzV&a,”‘aFieddn5]LP5‘—kx”twUz,?W:GUZytr.ge tu.HhJ]WC“‘VN, ’“:A#LVthwounMTD‘x, t?MAagxfiH
DyBAW*duNKwhyil
hathu#K:*Bcrgs’R)YNlDSksopqz!ExjPoQDymqldddTx)Sl
t Cu’2KPenyXfK[090pu#LPYFjsnkOu,53I—”x]I—
“&”[-I—me

t glp!fwndshat.he 2-I’UiU2—ythrooQ1—Egg inksN” h axAwihed5I#M?09K”—K
